
# Notebook Map: Relevance Evaluation Pipeline with Few-Shot + Agentic Enhancements

This table of contents provides a structured overview of the notebook, describing each section's purpose and how it fits into the workflow.

---

## 1. Quick Reference
- Overview of semantic versioning, few-shot prompting, and agentic conflict resolver.

## 2. Imports and Configuration
- Load required libraries and define configuration constants (e.g., few-shot parameters, log paths).

## 3. Core Utility Functions
- `verify_decision`: Ensures model decisions are consistent.
- `calculate_rule_confidence`: Computes rule-based confidence from criteria.
- `get_next_prompt_version`: Auto-increments semantic prompt version.
- `rebuild_few_shot_pool`: Builds balanced few-shot example set from log.
- `agentic_conflict_resolver`: Resolves discrepancies between model and rule evaluations.

## 4. Data Preparation
- Load PDF research papers from `data/raw`.
- Truncate text to fit LLM context window.

## 5. Few-Shot Prompt Building
- Retrieve high-confidence examples from log.
- Prepend examples to base relevance prompt.

## 6. Main Evaluation Loop
- Iterate through PDFs.
- Evaluate relevance using LLM.
- Apply rule-based scoring and hybrid confidence calculation.
- Flag documents for review when model vs. rule confidence diverges.

## 7. Logging and Versioning
- Append results to `prompt_evaluation_log.json`.
- Add `prompt_version`, `decision_source`, and `agentic_resolution` where applicable.

## 8. Visualization
- Display confidence distribution, relevance drift, and flagged discrepancy trends.

## 9. Enhancements (Appended)
- Additional functions and logging improvements appended at the end for optional use.

---


> **Note:** The following section explains core functionality and workflow.

<font size=10>**End-Term / Final Project**</font>

<font size=6>**AI for Research Proposal Automation**</font>

### **Business Problem - Create an AI system which will help you writing the research proposal aligning with the NOFO Document**
   



Meet Dr. Ian McCulloh, a seasoned research advisor and a leading voice in interdisciplinary science. Over the years, his lab has explored everything from AI for counterterrorism to social network analysis in neuroscience. His publication portfolio is vast, rich, and... chaotic.

When the National Institute of Mental Health released a new NOFO (Notice of Funding Opportunity) seeking innovative digital health solutions for mental health equity, Dr. Ian saw an opportunity. But there was a problem: despite his extensive work, none of his existing research was directly aligned with digital mental health interventions. And with NIH deadlines looming, manually identifying relevant angles and generating a competitive proposal would be a massive lift.

Dr. Ian wished for a smart assistant—one that could digest his past work, interpret the NOFO’s intent, spark new research directions, and even help draft proposal sections.

**The Challenge:**

Organizations and researchers often maintain large archives of publications and prior work. When responding to competitive grants—especially highly specific ones like NIH NOFOs—it becomes extremely difficult and time-consuming to:

1. Align past work with a new funding call.
2. Extract relevant expertise from unrelated projects.
3. Ideate novel, fundable research proposals tailored to complex criteria.
4. Generate high-quality text for grant submission that satisfies technical and scientific review criteria.

The manual effort to sift through dense research documents, match them to nuanced funding criteria, and write compelling, compliant proposals is labor-intensive, inconsistent, and prone to missed opportunities.

> **Note:** The following section explains core functionality and workflow.

### **The Case Study Approach**

**Objective**
1. Develop a generative AI-powered system using LLMs to automate and optimize the creation of NIH research proposals.
2. The tool will identify relevant prior research, generate aligned project ideas, and draft high-quality proposal content tailored to specific NOFO requirements.

**Given workflow:**

```mermaid
flowchart TD
    A[Read NOFO Document] --> B[Analyze Research Papers]
    B --> C[Filter Papers by Topic]
    C --> D[Generate Research Ideas]
    D --> E[Upload ideas to LLM]
    E --> F[Generate Proposal]
    F --> G[LLM Evaluation]
    G --> H{Meets criteria?}
    H -- NO --> F
    H -- YES --> I[Human Review]
    I --> J{Approved?}
    J -- NO --> F
    J -- YES --> K[Final Proposal]
```

**Enhanced workflow based on conversations with ChatGPT and Claude:**

```mermaid
flowchart TD
    A[Read NOFO Document] --> B[Extract Key Requirements & Evaluation Criteria]
    B --> C[Multi-Stage Paper Processing<br>(PyPDF → OCR)]
    C --> C1[Table Extraction]
    C --> C2[Figure Extraction (OCR + Captioning)]
    C1 --> D
    C2 --> D
    D[Hybrid Indexing & Filtering<br>(BM25 + Embeddings + Metadata)]
    D --> E[Agentic Research Synthesis<br>(Research Analyst + Proposal Writer + Compliance Checker)]
    E --> F[Generate Proposal Blueprint + Draft]
    F --> G[Multi-Criteria Evaluation<br/>(RAG + LLM-as-Judge + Guardrails)]
    G --> H{Score ≥ Threshold?}
    H -- NO --> I[Targeted Refinement Loop<br/>(Weakness-Specific Prompts)]
    I --> F
    H -- YES --> J[Caching + Checkpointing of Results]
    J --> K[Human Review Interface]
    K --> L{Approved?}
    L -- NO --> M[Capture Feedback & Return to Refinement]
    M --> F
    L -- YES --> N[Final Proposal + Deliverables]
    
    subgraph "Agentic Components"
        E1[Research Analyst Agent]
        E2[Proposal Writer Agent]
        E3[Compliance Checker Agent]
        E1 <--> E2
        E2 <--> E3
        E3 <--> E1
    end
```

> **Note:** The following section explains core functionality and workflow.

## **Setup - [2 Marks]**
---
<font color=Red>**Note:**</font> *1 marks is awarded for the Embedding Model configuration and 1 mark for the LLM Configuration.*

## Configuration and Setup

In [27]:
# @title Run this cell => Restart the session => Start executing the below cells **(DO NOT EXECUTE THIS CELL AGAIN)**

!pip install -q langchain==0.3.21 \
                huggingface_hub==0.29.3 \
                openai==1.68.2 \
                chromadb==0.6.3 \
                langchain-community==0.3.20 \
                langchain_openai==0.3.10 \
                lark==1.2.2\
                rank_bm25==0.2.2\
                numpy==2.2.4 \
                scipy==1.15.2 \
                scikit-learn==1.6.1 \
                transformers==4.50.0 \
                pypdf==5.4.0 \
                markdown-pdf==1.7 \
                tiktoken==0.9.0 \
                sentence_transformers==4.0.0 \
                torch==2.6.0 \
                pypdf \
                camelot-py \
                opencv-python \
                pytesseract \
                pdf2image


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [28]:
# !pip install -r ../requirements.txt
!pip freeze > requirements.txt

In [29]:
# Import required libraries for core functionality
import os
api_key = os.getenv("OPENAI_API_KEY")
base_url = os.getenv("OPENAI_BASE_URL")

In [30]:
# Import required libraries for core functionality
import warnings
warnings.filterwarnings('ignore')

# @title Defining the LLM Model - Use `gpt-4o-mini` Model
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

In [31]:
# ------------------------------------------------------------
# FEW-SHOT AND LOGGING CONFIG
# ------------------------------------------------------------
# These constants control how many examples are retrieved and the minimum confidence threshold.
# Modify here if you want more or fewer few-shot examples or to change the confidence cutoff.
FEW_SHOT_MAX_EXAMPLES = 4         # Total examples (balanced between relevant/irrelevant if possible)
# Minimum confidence threshold for including examples in few-shot prompting
MIN_CONFIDENCE_FOR_FEWSHOT = 70   # Minimum hybrid confidence (%) to consider for few-shot retrieval

# JSON log path
# Path to the cleaned JSON log file where prompt evaluation iterations are stored
LOG_PATH = "prompt_evaluation_log_cleaned.json"

# PDF Pre-Processing

In [32]:
# PDF Cleaning Step: Remove non-visual annotations (comments, links, form fields)
# Keeps images, diagrams, and visible callouts intact

# Import required libraries for core functionality
import os
import fitz  # PyMuPDF

def clean_pdf_annotations(input_path, output_path):
    """
    Strips non-visual annotations (comments, form fields, links) from a PDF
    while preserving visible images and diagrams.
    """
    doc = fitz.open(input_path)

    for page in doc:
        # Iterate over all annotations (not images)
        annot = page.first_annot
        while annot:
            next_annot = annot.next  # store reference to next annotation
            # Remove annotation object (highlights, comments, links)
            page.delete_annot(annot)
            annot = next_annot

    # Save cleaned PDF
    doc.save(output_path, garbage=4, deflate=True)
    doc.close()

# Clean NOFO file
input_pdf = "../data/NOFO.pdf"
cleaned_pdf = "../data/NOFO_cleaned.pdf"
clean_pdf_annotations(input_pdf, cleaned_pdf)
print(f"Cleaned PDF saved to: {cleaned_pdf}")

# Get de-annotated NOFO doc content using PyPDFLoader for evaluation step
from langchain.document_loaders import PyPDFLoader
pdf_file = "../data/NOFO_cleaned.pdf"
pdf_loader = PyPDFLoader(pdf_file);
NOFO_pdf = pdf_loader.load()

# Prepare output folder for de-annotated research papers
os.makedirs("data/raw", exist_ok=True)

# Set variables for de-annotating the research paper PDF collection 
source_dir = "../content"
output_dir = "data/raw"

# Loop through content folder, de-annotate each PDF, and save to a 'clean' output directory
for file_name in os.listdir(source_dir):
    if file_name.lower().endswith(".pdf"):
        input_pdf = os.path.join(source_dir, file_name)
        cleaned_pdf = os.path.join(output_dir, file_name.replace(".pdf", "_cleaned.pdf"))
        print(f"Cleaning annotations for: {file_name}")
        clean_pdf_annotations(input_pdf, cleaned_pdf)
        print(f"Cleaned PDF saved to: {cleaned_pdf}")
print("All research PDFs cleaned and saved in data/raw/")

Cleaned PDF saved to: ../data/NOFO_cleaned.pdf
Cleaning annotations for: cycon-final-draft.pdf
Cleaned PDF saved to: data/raw/cycon-final-draft_cleaned.pdf
Cleaning annotations for: Chat GPT Bias final w copyright.pdf
Cleaned PDF saved to: data/raw/Chat GPT Bias final w copyright_cleaned.pdf
Cleaning annotations for: Genetic_Algorithms_for_Prompt_Optimization.pdf
Cleaned PDF saved to: data/raw/Genetic_Algorithms_for_Prompt_Optimization_cleaned.pdf
Cleaning annotations for: DIVERSE_LLM_Dataset___IEEE_Big_Data.pdf
Cleaned PDF saved to: data/raw/DIVERSE_LLM_Dataset___IEEE_Big_Data_cleaned.pdf
Cleaning annotations for: Hashtag_Revival.pdf
Cleaned PDF saved to: data/raw/Hashtag_Revival_cleaned.pdf
Cleaning annotations for: FBI_Recruit_Hire_Final.pdf
Cleaned PDF saved to: data/raw/FBI_Recruit_Hire_Final_cleaned.pdf
Cleaning annotations for: Benson_MA491_NLP.pdf
Cleaned PDF saved to: data/raw/Benson_MA491_NLP_cleaned.pdf
Cleaning annotations for: Extreme Cohesion Darknet 20190815.pdf
Cleaned 

In [33]:
# Function to split cleaned text into 3000-token chunks with overlap for RAG.

import tiktoken
encoding = tiktoken.encoding_for_model("gpt-4o-mini")

def chunk_text(text, chunk_size=3000, overlap=200):
    tokens = encoding.encode(text)
    chunks = []
    for i in range(0, len(tokens), chunk_size - overlap):
        chunk_tokens = tokens[i:i+chunk_size]
        chunks.append(encoding.decode(chunk_tokens))
    return chunks

In [34]:
# Function to clean extracted text by:
# - removing headers/footers
# - removing noise
# - fixing multi-column layout issues

import re

def clean_extracted_text(text):
    """Remove noise (page numbers, headers, footers), merge hyphenated words,
    and flatten potential two-column layouts."""
    # Remove common noise patterns
    text = re.sub(r'\bPage \d+\b', '', text, flags=re.IGNORECASE)
    text = re.sub(r'\d+ of \d+', '', text, flags=re.IGNORECASE)

    # Remove repeated headers/footers (heuristic: lines repeated >3 times)
    lines = text.split('\n')
    freq = {}
    for line in lines:
        freq[line] = freq.get(line, 0) + 1
    lines = [line for line in lines if freq[line] <= 3]

    # Merge hyphenated words and normalize whitespace
    text = re.sub(r'-\n', '', '\n'.join(lines))
    text = re.sub(r'\n+', '\n', text)
    text = re.sub(r'\s{2,}', ' ', text)

    # Merge two-column text by pairing lines
    merged_lines = []
    lines = text.split('\n')
    for i in range(0, len(lines), 2):
        if i+1 < len(lines):
            merged_lines.append(lines[i] + " " + lines[i+1])
        else:
            merged_lines.append(lines[i])
    return "\n".join(merged_lines)

In [37]:
# Function to extract PDF text from cleaned research paper pdfs
from pypdf import PdfReader

def process_pdf_multistage(file_path):
    content = ""

    try:
        reader = PdfReader(file_path)
        for page in reader.pages:
            page_text = page.extract_text() or ""
            content += page_text
    except Exception as e:
        print(f"PyPDF extraction failed: {e}")

    # Clean text
    cleaned_text = clean_extracted_text(content)

    # Chunk text
    chunks = chunk_text(cleaned_text)

    return chunks

In [41]:
# Extract, clean, chunk, and store raw chunks for all research paper PDFs
import os
import json
from glob import glob

# Folder containing PDFs
pdf_folder = "data/raw"

# Output JSON file
output_json_path = "data/cleaned_chunked_papers.json"

# List to store results (each as a dict with id + chunks)
all_chunks = []

# Loop through all PDF files in the folder
for pdf_path in glob(os.path.join(pdf_folder, "*.pdf")):
    doc_name = os.path.basename(pdf_path)  # Use filename as ID
    print(f"Processing: {pdf_path}")
    try:
        chunks = process_pdf_multistage(pdf_path)  # uses your existing function
        all_chunks.append({
            "id": doc_name,
            "chunks": chunks
        })
    except Exception as e:
        print(f"Error processing {pdf_path}: {e}")

# Save results to JSON
with open(output_json_path, "w", encoding="utf-8") as f:
    json.dump(all_chunks, f, indent=2, ensure_ascii=False)

print(f"Saved cleaned + chunked text for {len(all_chunks)} PDFs to {output_json_path}")

Processing: data/raw/AAAI IAA CV_cleaned.pdf
Processing: data/raw/Sim of Decon_cleaned.pdf
Processing: data/raw/BotBuster___AAAI_cleaned.pdf
Processing: data/raw/Political_Networks_Conference_cleaned.pdf
Processing: data/raw/EmergencyResponseAI_cleaned.pdf
Processing: data/raw/FSS-19_paper_137_cleaned.pdf
Processing: data/raw/DIVERSE_LLM_Dataset___IEEE_Big_Data_cleaned.pdf
Processing: data/raw/Clustering_Analysis_of_Website_Usage_on_Twitter_during_the_COVID_19_Pandemic_cleaned.pdf
Processing: data/raw/Cohort_Optimization_Methods_SNAMS_2021_working_draft (4)_cleaned.pdf
Processing: data/raw/Lead-Azide_cleaned.pdf
Processing: data/raw/Knowing the Terrain_cleaned.pdf
Processing: data/raw/Leadership of Data Annotation 20180304v2_cleaned.pdf
Processing: data/raw/A_Complex_Network_Approach_to_Find_Latent_Terorrist_Communities_cleaned.pdf
Processing: data/raw/Designed Networks_cleaned.pdf
Processing: data/raw/Organizational risk using network analysis_cleaned.pdf
Processing: data/raw/LongNetV

In [43]:
# Add post-extraction GPT-enabled noise removal step

# Too resource intensive for full data set. Add later if needed.

# import json
# from openai import OpenAI

# # Initialize OpenAI client
# client = OpenAI()

# def semantic_clean_text(raw_text):
#     prompt = f"""
# You are a document cleaner. Extract ONLY the main body text from the following academic or technical document:
# - Remove page numbers, headers/footers
# - Remove title page, author affiliations, figure/table captions
# - Remove references/bibliography sections
# - Keep abstracts, introductions, main sections, and conclusions

# Document:
# \"\"\"{raw_text}\"\"\"

# Return only the cleaned text.
# """
#     response = client.responses.create(
#         model="gpt-4o-mini",
#         input=prompt,
#         max_output_tokens=4000
#     )
#     return response.output_text

# # --- Ingest cleaned + chunked data and post-process with GPT ---
# input_path = "data/cleaned_chunked_papers.json"
# output_path = "data/cleaned_gpt.json"

# # Load chunked data
# with open(input_path, "r", encoding="utf-8") as f:
#     chunked_data = json.load(f)

# # Prepare list for GPT-processed results
# gpt_cleaned_data = []

# # Loop through each document
# for record in chunked_data:
#     doc_id = record["id"]
#     gpt_chunks = []

#     print(f"Post-processing (GPT cleanup): {doc_id}")

#     # Apply GPT cleaning to each chunk
#     for chunk in record["chunks"]:
#         cleaned_chunk = semantic_clean_text(chunk)
#         gpt_chunks.append(cleaned_chunk)

#     # Store result
#     gpt_cleaned_data.append({
#         "id": doc_id,
#         "chunks": gpt_chunks
#     })

# # Save GPT-cleaned data
# with open(output_path, "w", encoding="utf-8") as f:
#     json.dump(gpt_cleaned_data, f, indent=2, ensure_ascii=False)

# print(f"Saved GPT post-processed chunks to {output_path}")


# Embedding Chunks

In [ ]:

# --- Chunk Scoring: BM25 + Cosine Similarity ---
# Hybrid scoring method combining BM25 ranking and embedding similarity.

# Placeholder: implement BM25 for tokenized chunks
from rank_bm25 import BM25Okapi
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')

def score_chunks(query, chunks):
    # BM25 scoring
    tokenized_chunks = [chunk.split() for chunk in chunks]
    bm25 = BM25Okapi(tokenized_chunks)
    bm25_scores = bm25.get_scores(query.split())

    # Embedding cosine similarity
    query_emb = model.encode([query])[0]
    chunk_embs = model.encode(chunks)
    cosine_scores = [np.dot(query_emb, ce) / (np.linalg.norm(query_emb) * np.linalg.norm(ce))
                     for ce in chunk_embs]

    # Combine (simple average or weighted)
    combined_scores = [(bm + cos) / 2 for bm, cos in zip(bm25_scores, cosine_scores)]
    return combined_scores


# --- Retrieval of Relevant Chunks ---
# Fetch top-N chunks by hybrid score for downstream RAG generation.

# def retrieve_relevant_chunks(query, chunks, top_k=5):
#     scores = score_chunks(query, chunks)
#     ranked = sorted(zip(chunks, scores), key=lambda x: x[1], reverse=True)
# #     return ranked[:top_k]

In [ ]:
# NOFO Match Evaluation Log Normaizing Function

def clean_json_log(json_path):
    """
    Cleans and normalizes older log entries for consistent structure:
    - Ensures relevant docs have confidence and discrepancy fields
    - Wraps irrelevant docs as list of titles (if stored as raw strings)
    """
# Import required libraries for core functionality
    import os, json
    if not os.path.exists(json_path):
        return

    with open(json_path, "r", encoding="utf-8") as f:
        try:
            data = json.load(f)
        except json.JSONDecodeError:
            print("Warning: JSON log corrupted. Starting fresh.")
            return

    updated = False
    for iteration in data:
        # Normalize relevant_documents
        normalized_relevant = []
        for doc in iteration.get("relevant_documents", []):
            # Convert string to dict if old format
            if isinstance(doc, str):
                normalized_relevant.append({
                    "title": doc,
                    "reasoning": "",
                    "model_confidence": 0,
                    "rule_confidence": 0,
                    "confidence_discrepancy": 0,
                    "flagged_for_review": False
                })
                updated = True
            else:
                # Ensure all expected fields exist
                doc.setdefault("model_confidence", 0)
                doc.setdefault("rule_confidence", 0)
                doc.setdefault("confidence_discrepancy", 0)
                doc.setdefault("flagged_for_review", False)
                normalized_relevant.append(doc)

        iteration["relevant_documents"] = normalized_relevant

        # Normalize irrelevant_documents to always be list of strings
        if isinstance(iteration.get("irrelevant_documents"), dict):
            iteration["irrelevant_documents"] = list(iteration["irrelevant_documents"].keys())
            updated = True

    # Save back only if modifications were made
    if updated:
        with open(json_path, "w", encoding="utf-8") as f:
            json.dump(data, f, indent=2, ensure_ascii=False)
        print("JSON log cleaned and normalized.")

In [ ]:
json_path = "prompt_evaluation_log_cleaned.json"

# ------------------------------------------------------------
# FEW-SHOT FALLBACK EXAMPLES
# ------------------------------------------------------------
FALLBACK_EXAMPLES = [
    (
        "Digital CBT for Adolescents",
        """{
  "criteria_results": {
    "domain_relevance": "YES - focuses on mental health digital interventions",
    "methodological_alignment": "YES - randomized controlled trial design",
    "theoretical_connection": "NO - lacks explicit framework reference",
    "practical_application": "YES - informs deployment in youth settings"
  },
  "decision": "RELEVANT",
  "confidence": "85",
  "summary": "This study evaluates a mobile CBT app for adolescents, showing significant reduction in anxiety and depression symptoms compared to control. It highlights engagement strategies relevant to NOFO objectives."
}"""
    ),
    (
        "Oncology Drug Delivery Review",
        """{
  "criteria_results": {
    "domain_relevance": "NO - focuses on oncology drug mechanisms",
    "methodological_alignment": "NO",
    "theoretical_connection": "NO",
    "practical_application": "NO"
  },
  "decision": "PAPER NOT RELATED TO TOPIC",
  "confidence": "0",
  "summary": null
}"""
    )
]


In [ ]:
# ------------------------------------------------------------
# FEW-SHOT RETRIEVAL + PROMPT BUILDER
# ------------------------------------------------------------
def get_few_shot_examples(
    json_path,
# Define configuration for few-shot example retrieval (number of examples)
    max_examples=FEW_SHOT_MAX_EXAMPLES,
# Minimum confidence threshold for including examples in few-shot prompting
    min_confidence=MIN_CONFIDENCE_FOR_FEWSHOT
):
    """
    Pulls balanced high-confidence examples from log or uses fallback if none found.
    """
# Import required libraries for core functionality
    import os, json, random

    examples = []

    # Attempt to pull from log
    if os.path.exists(json_path):
        with open(json_path, "r", encoding="utf-8") as f:
            try:
                data = json.load(f)
            except json.JSONDecodeError:
                data = []

        relevant_examples, irrelevant_examples = [], []

        for iteration in data:
            for doc in iteration.get("relevant_documents", []):
                hybrid_conf = max(doc.get("model_confidence", 0), doc.get("rule_confidence", 0))
                if hybrid_conf >= min_confidence:
                    relevant_examples.append((doc["title"], doc["reasoning"]))

            for doc in iteration.get("irrelevant_documents", []):
                irrelevant_examples.append((doc, "PAPER NOT RELATED TO TOPIC"))

        # Balance relevant and irrelevant (half and half)
        half = max_examples // 2
        random.shuffle(relevant_examples)
        random.shuffle(irrelevant_examples)
        selected_relevant = relevant_examples[:half]
        selected_irrelevant = irrelevant_examples[:half]
        examples = selected_relevant + selected_irrelevant

    # Fallback if no examples found
    if not examples:
        print("No high-confidence examples found. Using fallback seed examples.")
        examples = FALLBACK_EXAMPLES[:max_examples]

    return examples


def build_prompt_with_examples(topic, base_prompt, examples):
    """
    Prepend few-shot examples (from log or fallback) to the base prompt.
    """
    if not examples:
        return base_prompt

    examples_str = "\n\n".join(
        [f"Example ({title}):\n{reasoning}" for title, reasoning in examples]
    )

    return f"""
You are a research grant specialist evaluating research papers for relevance to NIH NOFO objectives: {topic}.

Below are examples of prior evaluations for context:
{examples_str}

Now evaluate the following paper using the same structure and logic:

{base_prompt}
"""


> **Note:** The following section explains core functionality and workflow.

## **Step 1: Topic Extraction - [3 Marks]**

> **Read the NOFO doc and identify the topic for which the funding is to be given.**
---
<font color=Red>**Note:**</font> *2 marks are awarded for the prompt and 1 mark for the successful completion of this section, including debugging or modifying the code if necessary.*
   

**TASK:** Write an LLM prompt to extract the Topic for what the funding is been provided, from the NOFO document, Ask the LLM to respond back with the topic name only and nothing else.

In [ ]:
# Topic extraction prompt
topic_extraction_prompt = f"""
You are a research grant specialist with expertise in analyzing NIH funding announcements and extracting key research priorities.

Your task: Analyze this NOFO document from the National Institute of Mental Health (NIMH) to identify the PRIMARY funding topic.

The document may describe multiple research areas, objectives, and priorities. Extract the single overarching topic that encompasses the main focus of this funding opportunity.

Return ONLY the primary topic in 3-8 words. No explanations, descriptions, or additional text.

Document:
{NOFO_pdf[0].page_content}
"""

In [ ]:
# Finding the topic for which the Funding is been given
topic_extraction = llm.invoke(topic_extraction_prompt)
topic = topic_extraction.content
topic

# Note: Multiple iterations of the above prompt yielded 'Digital mental health interventions' from both Open AI and Claude.

> **Note:** The following section explains core functionality and workflow.

## **Step 2: Research Paper Relevance Assessment - [3 Marks]**
> **Analyze all the Research Papers and filter out the research papers based on the topic of NOFO**
---
<font color=Red>**Note:**</font> *2 marks are awarded for the prompt and 1 mark for the successful completion of this section, including debugging or modifying the code if necessary.*

> **Note:** The following section explains core functionality and workflow.

**TASK:** Write an Prompt which can be used to analyze the relevance of the provided research paper in relation to the topic outlined in the NOFO (Notice of Funding Opportunity) document. Determine whether the research aligns with the goals, objectives, and funding criteria specified in the NOFO. Additionally, assess whether the research paper can be used to support or develop a viable project idea that fits within the scope of the funding opportunity.

<br>

**Note:** If the paper does **not** significantly relate to the topic—by domain, method, theory, or application ask the LLM to return: **"PAPER NOT RELATED TO TOPIC"**


<br>

Ask the LLM to respond in the below specified structure:

```
### Output Format:
"summary": "<summary of the paper under 300 words, or return: PAPER NOT RELATED TO TOPIC>"

```

In [ ]:
relevance_prompt_a = f"""
You are a research grant specialist evaluating research papers for relevance to NIH NOFO objectives: {topic}.

Evaluate the paper step-by-step against these criteria:
1. Domain relevance (mental health, digital health, intervention effectiveness)
2. Methodological alignment (clinical trials, user engagement studies, technology development)
3. Theoretical connection (frameworks, evidence, insights for intervention design/implementation)
4. Practical application (supports development or testing of digital mental health interventions)

Instructions:
- For EACH criterion, respond YES or NO and justify briefly.
- A paper is RELEVANT if at least ONE criterion is YES.
- Assign a confidence score (0–100%) to the RELEVANT decision, based on how strongly the paper meets the criteria (higher = more confident relevance).
- If RELEVANT: provide a <300-word summary focused on digital mental health intervention insights.
- If NOT RELEVANT: return exactly "PAPER NOT RELATED TO TOPIC".

Output format (JSON):
{{
  "criteria_results": {{
    "domain_relevance": "YES/NO - justification",
    "methodological_alignment": "YES/NO - justification",
    "theoretical_connection": "YES/NO - justification",
    "practical_application": "YES/NO - justification"
  }},
  "decision": "RELEVANT" or "PAPER NOT RELATED TO TOPIC",
  "confidence": "<integer between 0 and 100>",
  "summary": "<summary text or null>"
}}

### Paper content:
"""

In [ ]:
def build_prompt_with_examples(topic, base_prompt, examples):
    """
    Assemble few-shot prompt:
    - Inserts prior examples (formatted) before evaluation instructions
    """
    examples_str = "\n\n".join(
        [f"Example ({title}):\n{reasoning}" for title, reasoning in examples]
    )

    prompt = f"""
You are a research grant specialist evaluating research papers for relevance to NIH NOFO objectives: {topic}.

Below are examples of prior evaluations for context:
{examples_str}

Now evaluate the following paper using the same structure and logic:

{base_prompt}
"""
    return prompt


In [ ]:
# --- Few-shot setup ---
# Path to the cleaned JSON log file where prompt evaluation iterations are stored
clean_json_log(LOG_PATH)
# Path to the cleaned JSON log file where prompt evaluation iterations are stored
few_shot_examples = get_few_shot_examples(LOG_PATH)
prompt_with_examples = build_prompt_with_examples(topic, relevance_prompt_a, few_shot_examples)

# Import required libraries for core functionality
import os
# Import required libraries for core functionality
import json
# Import required libraries for core functionality
import random
# Import required libraries for core functionality
import tiktoken
from datetime import datetime
# Import required libraries for core functionality
import re
# Import required libraries for core functionality
import matplotlib.pyplot as plt

# ------------------------------------------------------------
# CONFIGURATION SECTION
# ------------------------------------------------------------
# These variables let you control behavior without editing main logic.

TEST_MODE = True                  # If True, process only a subset of files for quick iteration
TEST_SAMPLE_SIZE = 60          # How many files to evaluate in test mode
STRATIFY = True                   # If True, attempt stratified sampling (balanced categories)
DISCREPANCY_THRESHOLD = 20        # Flag difference (%) between model vs rule confidence for review

# Prior classification data (if available) can guide stratified sampling
# e.g., after first run, categorize known relevant/irrelevant papers
prior_classification = {
    "relevant": [],   # fill with filenames identified as relevant
    "irrelevant": [], # fill with filenames identified as irrelevant
    "unknown": []     # files not yet evaluated or borderline
}

# ------------------------------------------------------------
# LOGGING FUNCTION
# ------------------------------------------------------------
def log_prompt_iteration(
    json_path,
    prompt,
    relevant_docs_with_reasoning,
    irrelevant_docs,
):
    """
    Append this iteration's results (prompt + classified documents) to a master JSON log.

    Rationale:
    - Allows longitudinal analysis of prompt versions and performance trends
    - Facilitates reproducibility for future audit or review
    """
    iteration_id = len(json.load(open(json_path))) + 1 if os.path.exists(json_path) else 1
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    entry = {
        "iteration_id": iteration_id,
        "timestamp": timestamp,
        "prompt": prompt,
        "relevant_documents": relevant_docs_with_reasoning,
        "irrelevant_documents": irrelevant_docs
    }

    # Load existing log or create a new one
    if os.path.exists(json_path):
        with open(json_path, "r", encoding="utf-8") as f:
            try:
                data = json.load(f)
            except json.JSONDecodeError:
                data = []
    else:
        data = []

    data.append(entry)
    with open(json_path, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=2, ensure_ascii=False)

    print(f"Logged iteration {iteration_id} to {json_path}")


# ------------------------------------------------------------
# SELF-CHECK FUNCTION
# ------------------------------------------------------------
# Function: Self-check verification of LLM decision consistency
def verify_decision(llm, reasoning_output):
    """
    Performs a secondary verification pass using the model itself:
    - Inputs the full reasoning text
    - Asks for binary 'YES' or 'NO' confirmation of relevance

    Rationale:
    - Adds a lightweight consistency check
    - Reduces false positives where reasoning contradicts final label
    """
    verification_prompt = f"""
You are verifying the relevance decision based on the following evaluation:

{reasoning_output}

Only answer with 'YES' if the decision should be considered relevant, or 'NO' if not relevant.
    """
# Call the LLM with the prepared prompt and truncated paper content
    verification_response = llm.invoke(verification_prompt)
    return "YES" in verification_response.content.upper()


# ------------------------------------------------------------
# RULE-DERIVED CONFIDENCE FUNCTION
# ------------------------------------------------------------
# Function: Compute rule-based confidence score based on YES criteria count
def calculate_rule_confidence(criteria_results):
    """
    Computes deterministic confidence score based on count of YES criteria.

    Mapping (transparent to stakeholders):
    - 0 YES = 0%
    - 1 YES = 50%
    - 2 YES = 70%
    - 3 YES = 85%
    - 4 YES = 95%

    Rationale:
    - Provides reproducible baseline independent of model's self-estimation
    - Useful for auditing or hybrid scoring strategies
    """
    yes_count = sum(1 for v in criteria_results.values() if v.upper().startswith("YES"))
    if yes_count == 0:
        return 0
    elif yes_count == 1:
        return 50
    elif yes_count == 2:
        return 70
    elif yes_count == 3:
        return 85
    else:
        return 95


# ------------------------------------------------------------
# STRATIFIED FILE SAMPLING FUNCTION
# ------------------------------------------------------------
def get_files_to_process(path):
    """
    Builds file list for processing:
    - Uses full dataset if TEST_MODE = False
    - Otherwise randomly samples TEST_SAMPLE_SIZE
    - If STRATIFY = True and prior classifications exist, balances sample
      across relevant/irrelevant/unknown groups

    Rationale:
    - Rapid iterations on representative subsets improve prompt tuning speed
    - Stratification ensures diverse coverage (avoids subset bias)
    """
    all_files = [f for f in os.listdir(path) if f.endswith('.pdf')]

    if not TEST_MODE:
        return all_files

    if STRATIFY and any(prior_classification.values()):
        files_to_process = []
        groups = ['relevant', 'irrelevant', 'unknown']
        quota = max(1, TEST_SAMPLE_SIZE // len(groups))

        for group in groups:
            pool = [f for f in all_files if f in prior_classification[group]]
            if pool:
                files_to_process.extend(random.sample(pool, min(quota, len(pool))))

        # Fill remaining slots randomly if stratified pool is too small
        remaining = TEST_SAMPLE_SIZE - len(files_to_process)
        if remaining > 0:
            leftover_pool = list(set(all_files) - set(files_to_process))
            files_to_process.extend(random.sample(leftover_pool, min(remaining, len(leftover_pool))))
    else:
        files_to_process = random.sample(all_files, min(TEST_SAMPLE_SIZE, len(all_files)))

    return files_to_process


# ------------------------------------------------------------
# MAIN LOOP: CLASSIFY DOCUMENTS
# ------------------------------------------------------------
path = "data/raw"
files_to_process = get_files_to_process(path)

documents = []            # Stores relevant docs with reasoning + confidences
irrelevant_docs_list = []  # Stores filenames for irrelevant docs
total_files = len(files_to_process)

# Configure tokenization for the specified LLM to manage context window limits
encoding = tiktoken.encoding_for_model("gpt-4o-mini")
MAX_TOKENS = 300000

# MAX_TOKENS = 127500
# Purpose:
# Sets an internal limit you use for chunking or retrieval pipelines to ensure total token usage stays within the model’s maximum context window (≈128k tokens for GPT‑4o‑mini).
# Impact on your project:

# Document Chunking: Controls how big each chunk (or combined chunks) can be before sending them to the LLM.
# Hybrid Indexing: Influences how much content from BM25 + embeddings can be merged into a single query.
# Agent Orchestration: Each agent’s context window (Research, Proposal Writer, Compliance) will be limited by this ceiling.
# Proposal Generation: Prevents oversized prompts when combining NOFO sections, extracted tables/figures, and retrieved research papers.
# Refinement Loops: Ensures each pass fits in context even as drafts get longer (important for iterative proposals).

# 3. Why It Matters in Your Flow
# Your pipeline does multi-modal RAG (text + tables + figure captions). Without careful token counting:
# Combined context from NOFO + papers + tables could easily exceed the limit.
# Hybrid retrieval (BM25 + embeddings) may bring in redundant or overlapping chunks unless you budget tokens.
# Setting MAX_TOKENS just below the model’s hard limit (127.5k vs. 128k) is a safety margin—avoids hard errors during synthesis or evaluation.

progress_cnt = 1
relevant_papers_count = 0
irrelevant_papers_count = 0

for filename in files_to_process:
    file_path = os.path.join(path, filename)

    try:
        # -------------------------
        # Load PDF and prepare text
        # -------------------------
# Load PDF files and extract content using PyPDFLoader
        docs = PyPDFLoader(file_path, mode="single").load()
        pages = docs[0].page_content

        # Token management: truncate paper text to fit model context window
        available_tokens = MAX_TOKENS - len(encoding.encode(prompt_with_examples))
        truncated_pages = encoding.decode(encoding.encode(pages)[:available_tokens])
        full_prompt = prompt_with_examples + truncated_pages

        # -------------------------
        # Primary LLM evaluation
        # -------------------------
# Call the LLM with the prepared prompt and truncated paper content
        response = llm.invoke(full_prompt)
# Provide progress feedback to user on processing status
        print(f"Successfully processed: {progress_cnt}/{total_files}")
        progress_cnt += 1

        # -------------------------
        # Self-check verification
        # -------------------------
        is_relevant = verify_decision(llm, response.content)

        if not is_relevant or "PAPER NOT RELATED TO TOPIC" in response.content:
            irrelevant_papers_count += 1
            irrelevant_docs_list.append(filename)
            continue

        # -------------------------
        # Parse JSON-like output
        # -------------------------
        try:
            parsed_json = json.loads(response.content)
        except json.JSONDecodeError:
            # Fallback regex parse if model wraps JSON in text
            json_match = re.search(r"\{.*\}", response.content, re.DOTALL)
            parsed_json = json.loads(json_match.group(0)) if json_match else {}

        # -------------------------
        # Extract confidences
        # -------------------------
        # Model-estimated confidence (direct from LLM)
        model_confidence = int(parsed_json.get("confidence", 0)) if parsed_json else None

        # Rule-derived confidence (count of YES answers)
        rule_confidence = 0
        if "criteria_results" in parsed_json:
            rule_confidence = calculate_rule_confidence(parsed_json["criteria_results"])

        # Discrepancy between two confidences
        discrepancy = None
        flagged = False
        if model_confidence is not None:
            discrepancy = abs(model_confidence - rule_confidence)
            flagged = discrepancy > DISCREPANCY_THRESHOLD  # auto-flag if > threshold

        # Store structured result
        documents.append({
            'title': filename,
            'file_path': file_path,
            'llm_reasoning': response.content,
            'model_confidence': model_confidence,
            'rule_confidence': rule_confidence,
            'confidence_discrepancy': discrepancy,
            'flagged_for_review': flagged
        })
        relevant_papers_count += 1

    except Exception as e:
        print(f"!!! Error processing {filename}: {str(e)}")


# ------------------------------------------------------------
# SUMMARY OUTPUT
# ------------------------------------------------------------
print("=" * 50)
print(f"Relevant Papers: {relevant_papers_count}/{total_files}")
print(f"Irrelevant Papers: {irrelevant_papers_count}/{total_files}")

print("\nList of relevant papers:")
for doc in documents:
    print(f"\nTitle: {doc['title']}")
    print(f"Model Confidence: {doc['model_confidence']}")
    print(f"Rule Confidence: {doc['rule_confidence']}")
    print(f"Discrepancy: {doc['confidence_discrepancy']} (Flagged: {doc['flagged_for_review']})")
    print(f"Reasoning (truncated): {doc['llm_reasoning'][:500]}...")


# ------------------------------------------------------------
# LOGGING: MASTER + FLAGGED
# ------------------------------------------------------------
# Prepare relevant docs with reasoning for main log
relevant_docs_with_reasoning = [
    {
        "title": doc['title'],
        "reasoning": doc['llm_reasoning'],
        "model_confidence": doc['model_confidence'],
        "rule_confidence": doc['rule_confidence'],
        "confidence_discrepancy": doc['confidence_discrepancy'],
        "flagged_for_review": doc['flagged_for_review']
    }
    for doc in documents
]

# Log all results
log_prompt_iteration(
    json_path="prompt_evaluation_log_cleaned.json",
    prompt=prompt_with_examples,
    relevant_docs_with_reasoning=relevant_docs_with_reasoning,
    irrelevant_docs=irrelevant_docs_list,
)

# Save flagged docs separately for manual review queue
flagged_docs = [doc for doc in relevant_docs_with_reasoning if doc["flagged_for_review"]]
if flagged_docs:
    with open("flagged_for_review.json", "w", encoding="utf-8") as f:
        json.dump(flagged_docs, f, indent=2, ensure_ascii=False)
    print(f"Saved {len(flagged_docs)} flagged documents to flagged_for_review.json")


# ------------------------------------------------------------
# VISUALIZATION
# ------------------------------------------------------------
# Compare model vs rule confidence distributions
# model_conf = [doc['model_confidence'] for doc in documents if doc['model_confidence'] is not None]
# rule_conf = [doc['rule_confidence'] for doc in documents]

# if model_conf and rule_conf:
#     # Histogram: distribution comparison
# # Visualization: Generate plots for confidence distributions or trends
#     plt.figure(figsize=(6, 4))
# # Visualization: Generate plots for confidence distributions or trends
#     plt.hist(model_conf, bins=10, alpha=0.5, label="Model Confidence")
# # Visualization: Generate plots for confidence distributions or trends
#     plt.hist(rule_conf, bins=10, alpha=0.5, label="Rule Confidence")
# # Visualization: Generate plots for confidence distributions or trends
#     plt.legend()
# # Visualization: Generate plots for confidence distributions or trends
#     plt.title("Confidence Distribution")
# # Visualization: Generate plots for confidence distributions or trends
#     plt.xlabel("Confidence (%)")
# # Visualization: Generate plots for confidence distributions or trends
#     plt.ylabel("Count")
# # Visualization: Generate plots for confidence distributions or trends
#     plt.show()

#     # Scatterplot: identify discrepancies visually
# # Visualization: Generate plots for confidence distributions or trends
#     plt.figure(figsize=(6, 6))
#     colors = ["red" if doc['flagged_for_review'] else "blue" for doc in documents]
# # Visualization: Generate plots for confidence distributions or trends
#     plt.scatter(rule_conf, model_conf, c=colors, alpha=0.6)
# # Visualization: Generate plots for confidence distributions or trends
#     plt.axline((0, 0), slope=1, color="gray", linestyle="--")  # perfect agreement line
# # Visualization: Generate plots for confidence distributions or trends
#     plt.title("Model vs Rule Confidence (Flagged in Red)")
# # Visualization: Generate plots for confidence distributions or trends
#     plt.xlabel("Rule-derived Confidence (%)")
# # Visualization: Generate plots for confidence distributions or trends
#     plt.ylabel("Model-estimated Confidence (%)")
# # Visualization: Generate plots for confidence distributions or trends
#     plt.show()



# ------------------------------------------------------------
# AUTO-UPDATE PRIOR CLASSIFICATION FOR STRATIFIED SAMPLING
# ------------------------------------------------------------
# After each run, update prior_classification with filenames categorized this iteration.

for doc in relevant_docs_with_reasoning:
    if doc["title"] not in prior_classification["relevant"]:
        prior_classification["relevant"].append(doc["title"])

for doc in irrelevant_docs_list:
    if doc not in prior_classification["irrelevant"]:
        prior_classification["irrelevant"].append(doc)

# Remove duplicates if any
for key in prior_classification:
    prior_classification[key] = list(set(prior_classification[key]))

print("Updated prior_classification for future stratified sampling.")



# ------------------------------------------------------------
# SUMMARY OF PRIOR CLASSIFICATION (TRACKED IN LOG)
# ------------------------------------------------------------
summary_counts = {
    "relevant_count": len(prior_classification["relevant"]),
    "irrelevant_count": len(prior_classification["irrelevant"]),
    "unknown_count": len(prior_classification["unknown"])
}

# Append summary to last log entry
# Path to the cleaned JSON log file where prompt evaluation iterations are stored
with open(LOG_PATH, "r+", encoding="utf-8") as f:
    data = json.load(f)
    data[-1]["prior_classification_summary"] = summary_counts
    f.seek(0)
    json.dump(data, f, indent=2, ensure_ascii=False)
    f.truncate()

print(f"Classification summary appended to log: {summary_counts}")

# ------------------------------------------------------------
# VISUALIZATION: EVOLUTION OF CLASSIFICATIONS OVER TIME
# ------------------------------------------------------------
# Create trend lines for relevant/irrelevant/unknown counts across iterations
# iterations = list(range(1, len(data)+1))
# relevant_counts = [entry.get("prior_classification_summary", {}).get("relevant_count", 0) for entry in data]
# irrelevant_counts = [entry.get("prior_classification_summary", {}).get("irrelevant_count", 0) for entry in data]
# unknown_counts = [entry.get("prior_classification_summary", {}).get("unknown_count", 0) for entry in data]

# # Visualization: Generate plots for confidence distributions or trends
# plt.figure(figsize=(8, 5))
# # Visualization: Generate plots for confidence distributions or trends
# plt.plot(iterations, relevant_counts, label="Relevant", marker='o')
# # Visualization: Generate plots for confidence distributions or trends
# plt.plot(iterations, irrelevant_counts, label="Irrelevant", marker='o')
# # Visualization: Generate plots for confidence distributions or trends
# plt.plot(iterations, unknown_counts, label="Unknown", marker='o')
# # Visualization: Generate plots for confidence distributions or trends
# plt.title("Classification Evolution Over Iterations")
# # Visualization: Generate plots for confidence distributions or trends
# plt.xlabel("Iteration")
# # Visualization: Generate plots for confidence distributions or trends
# plt.ylabel("Count")
# # Visualization: Generate plots for confidence distributions or trends
# plt.legend()
# # Visualization: Generate plots for confidence distributions or trends
# plt.grid(True)
# # Visualization: Generate plots for confidence distributions or trends
# plt.show()



# ------------------------------------------------------------
# ALERT: FLAGGED DISCREPANCIES TREND ANALYSIS
# ------------------------------------------------------------
# Count flagged docs per iteration
flagged_counts = [len([doc for doc in entry.get("relevant_documents", []) if doc.get("flagged_for_review")]) for entry in data]

# Define threshold (e.g., >30% flagged across last 3 iterations)
if len(flagged_counts) >= 3:
    recent_flagged = flagged_counts[-3:]
    total_recent = [len(entry.get("relevant_documents", [])) for entry in data[-3:]]
    flagged_ratios = [fc/tr if tr > 0 else 0 for fc, tr in zip(recent_flagged, total_recent)]

    avg_flagged_ratio = sum(flagged_ratios) / len(flagged_ratios)

    if avg_flagged_ratio > 0.3:
        print(f"WARNING: High flagged discrepancy rate detected over last 3 iterations: {avg_flagged_ratio*100:.1f}%")
    else:
        print(f"Flagged discrepancy rate stable: {avg_flagged_ratio*100:.1f}% over last 3 iterations.")
else:
    print("Not enough iterations yet for flagged discrepancy trend analysis.")



# ------------------------------------------------------------
# ALERT: FLAGGED DISCREPANCIES TREND ANALYSIS + LOGGING
# ------------------------------------------------------------
# Count flagged docs per iteration
flagged_counts = [len([doc for doc in entry.get("relevant_documents", []) if doc.get("flagged_for_review")]) for entry in data]

alert_status = "No alert"
if len(flagged_counts) >= 3:
    recent_flagged = flagged_counts[-3:]
    total_recent = [len(entry.get("relevant_documents", [])) for entry in data[-3:]]
    flagged_ratios = [fc/tr if tr > 0 else 0 for fc, tr in zip(recent_flagged, total_recent)]

    avg_flagged_ratio = sum(flagged_ratios) / len(flagged_ratios)

    if avg_flagged_ratio > 0.3:
        alert_status = f"WARNING: High flagged discrepancy rate: {avg_flagged_ratio*100:.1f}% over last 3 iterations"
        print(alert_status)
    else:
        alert_status = f"Stable flagged discrepancy rate: {avg_flagged_ratio*100:.1f}% over last 3 iterations"
        print(alert_status)
else:
    alert_status = "Not enough iterations for trend analysis"
    print(alert_status)

# Log alert status into JSON (append to last entry)
# Path to the cleaned JSON log file where prompt evaluation iterations are stored
with open(LOG_PATH, "r+", encoding="utf-8") as f:
    data = json.load(f)
    data[-1]["flagged_discrepancy_alert"] = alert_status
    f.seek(0)
    json.dump(data, f, indent=2, ensure_ascii=False)
    f.truncate()

cleaned_log = clean_and_normalize_log(LOG_PATH)

# ------------------------------------------------------------
# VISUALIZATION: FLAGGED DISCREPANCY DRIFT OVER TIME
# ------------------------------------------------------------
# Compute ratio of flagged relevant docs per iteration
# flagged_ratios_all = []
# for entry in data:
#     relevant_docs = entry.get("relevant_documents", [])
#     total = len(relevant_docs)
#     flagged = len([doc for doc in relevant_docs if doc.get("flagged_for_review")])
#     ratio = (flagged / total) if total > 0 else 0
#     flagged_ratios_all.append(ratio)

# # Plot drift over time
# # Visualization: Generate plots for confidence distributions or trends
# plt.figure(figsize=(8, 4))
# # Visualization: Generate plots for confidence distributions or trends
# plt.plot(iterations, [r*100 for r in flagged_ratios_all], marker='o', color='red')
# # Visualization: Generate plots for confidence distributions or trends
# plt.title("Flagged Discrepancy Ratio Over Time")
# # Visualization: Generate plots for confidence distributions or trends
# plt.xlabel("Iteration")
# # Visualization: Generate plots for confidence distributions or trends
# plt.ylabel("Flagged Ratio (%)")
# # Visualization: Generate plots for confidence distributions or trends
# plt.grid(True)
# # Visualization: Generate plots for confidence distributions or trends
# plt.axhline(30, color='gray', linestyle='--', label='Alert Threshold (30%)')
# # Visualization: Generate plots for confidence distributions or trends
# plt.legend()
# # Visualization: Generate plots for confidence distributions or trends
# plt.show()


In [ ]:
import json
import os
from pypdf import PdfReader
import tiktoken

# --- Tokenization setup ---
encoding = tiktoken.encoding_for_model("gpt-4o-mini")
EXTRACTION_BUDGET = 300000  # ~20% buffer below max context window

def count_tokens(text):
    return len(encoding.encode(text))

# --- Load matching papers from JSON log ---
def load_matched_papers(json_path, pdf_folder="data/raw"):
    with open(json_path, "r") as f:
        data = json.load(f)
    
    last_iteration = data[-1]
    relevant_docs = last_iteration.get("relevant_documents", [])
    
    file_paths = []
    for doc in relevant_docs:
        title = doc["title"]
        pdf_path = os.path.join(pdf_folder, title)
        if os.path.exists(pdf_path):
            file_paths.append(pdf_path)
        else:
            print(f"Warning: {pdf_path} not found. Skipping.")
    return file_paths



# --- Process all matched papers ---
def process_matched_papers(json_path, pdf_folder="content"):
    matched_files = load_matched_papers(json_path, pdf_folder)
    results = {}
    for file_path in matched_files:
        print(f"Processing: {os.path.basename(file_path)}")
        results[os.path.basename(file_path)] = process_pdf_multistage(file_path)
    return results

# Example usage
matched_content = process_matched_papers("prompt_evaluation_log_cleaned.json", pdf_folder="data/raw")
print(matched_content.keys())


In [ ]:
for filename, content in matched_content.items():
    print(f"\n--- {filename} ---\n")
    print(content[:1000])  # print first 1000 chars to avoid overwhelming output
    print("\n-------------------\n")

> **Note:** The following section explains core functionality and workflow.

## **Step 3: Proposal Ideation Based on Filtered Research - [4 marks]**
> **Use the filtered papers, to generate ideas for the Reseach Proposal.**
---
<font color=Red>**Note:**</font> *2 marks are awarded for the prompt, 1 mark for the Generating Idea and 1 mark for fetching file path of chosen idea along with successful completion of this section, including debugging or modifying the code if necessary.*

> **Note:** The following section explains core functionality and workflow.

**TASK:** Write an Prompt which can be used to generate 5 ideas for the Research Proposal, each idea should consist:

1. **Idea X:** [Concise Title of the Project Idea]  \n
2. **Description:** [Brief and targeted description summarizing the objectives, innovative elements, scientific rationale, and anticipated impact.]  \n
3. **Citation:** [Author(s), Year or Paper Title]  \n
4. **NOFO Alignment:** [List two or more specific NOFO requirements that this idea directly addresses]  \n
5. **File Path of the Research Paper:** [Exact file path, ending in .pdf]

- Use the Delimiter `---` for defining the structure of the sample outputs in the prompt





> **Note:** The following section explains core functionality and workflow.

#### Generating 5 Ideas

In [ ]:
# Note to self: Be sure to add additional details from page linked in the NOFO pdf
# Also need to include constraints, e.g., "Digital health test beds that leverage well-established 
# digital health platforms to optimize evidence-based digital mental health interventions"

gen_idea_prompt = f"""


<WRITE YOUR PROMPT HERE>


"""

In [ ]:
ideas = llm.invoke(gen_idea_prompt)

In [ ]:
from IPython.display import Markdown, display
display(Markdown(ideas.content))

> **Note:** The following section explains core functionality and workflow.

#### Choosing 1 Idea and fetching details

In [ ]:
# Modify the idea_number for choosing the different idea
idea_number = 5   # change the number if you wish to choose and generate the research proposal for another idea
chosen_idea = ideas.content.split("---")[idea_number]

In [ ]:
# Import required libraries for core functionality
import re

# Use a regular expression to find the file path of the research paper

pattern = r"File Path of the Research Paper:\*\*\s*(.+?)\n"
# If you are unable to extract the file path successfully using this pattern, use the `ChatGPT` or any other LLM to find the pattern that works for you, simply provide the LLM the sample response of your whole ideas and ask the LLM to generate the regex patterm for extracting the "File Path of the Research Paper"

match = re.search(pattern, chosen_idea)

if match:
  idea_generated_from_research_paper = match.group(1).strip()
  print("Filepath : ", idea_generated_from_research_paper)
else:
  print("File Path of the Research Paper not found in the chosen idea.")

> **Note:** The following section explains core functionality and workflow.

## **Step 4: Proposal Blueprint Preparation - [3 Marks]**

> **Select appropriate research ideas for the proposal and supply 'Sample Research Proposals' as templates to the LLM to support the generation of the final proposal.**
---   
<font color=Red>**Note:**</font> *2 marks are awarded for the prompt and 1 mark for the successful completion of this section, including debugging or modifying the code if necessary.*

**TASK:** Write an Prompt which can be used to generate the Research Proposal.

The prompt should be able to craft a research proposal based on the sample research proposal template, using one of the ideas generated above. The proposal should include references to the actual research papers from which the ideas are derived and should align well with the NOFO documents.

In [ ]:
# Here we need to add the full papers instead of the summary
# Load PDF files and extract content using PyPDFLoader
chosen_idea_rp = PyPDFLoader(idea_generated_from_research_paper, mode="single").load()

# Loading the sample research proposal template
# Load PDF files and extract content using PyPDFLoader
research_proposal_template = PyPDFLoader(" <Path of Research Proposal Template> ", mode="single").load()

In [ ]:
import json
import os
from pypdf import PdfReader
import camelot
import pytesseract
from pdf2image import convert_from_path
import tiktoken

# --- Tokenization setup ---
encoding = tiktoken.encoding_for_model("gpt-4o-mini")
MAX_TOKENS = 127500          # total model context window
EXTRACTION_BUDGET = 100000   # reserve ~20% for prompts/response

def count_tokens(text):
    """Count tokens using tiktoken encoding."""
    return len(encoding.encode(text))

# --- Load matching papers from JSON log ---
def load_matched_papers(json_path, pdf_folder="content"):
    """
    Extract list of relevant document file paths from the latest JSON iteration.
    """
    with open(json_path, "r") as f:
        data = json.load(f)
    
    # Take the last iteration's relevant_documents
    last_iteration = data[-1]
    relevant_docs = last_iteration.get("relevant_documents", [])
    
    # Build file paths for each relevant doc (assumes they exist in pdf_folder)
    file_paths = []
    for doc in relevant_docs:
        title = doc["title"]
        pdf_path = os.path.join(pdf_folder, title)
        if os.path.exists(pdf_path):
            file_paths.append(pdf_path)
        else:
            print(f"Warning: {pdf_path} not found. Skipping.")
    return file_paths

# --- Stage 1 & 2: Text + Table extraction ---
def extract_text_and_tables(file_path, token_budget):
    """Extract text and tables within token budget."""
    content = ""
    token_count = 0

    # Stage 1: PyPDF text extraction
    try:
        reader = PdfReader(file_path)
        for page in reader.pages:
            page_text = page.extract_text() or ""
            token_count += count_tokens(page_text)
            if token_count > token_budget:
                print(f"Token budget reached during text extraction: {file_path}")
                break
            content += page_text
    except Exception as e:
        print(f"PyPDF extraction failed: {e}")

    # Stage 2: Table extraction (Camelot)
    # try:
    #     tables = camelot.read_pdf(file_path, pages='all')
    #     for table in tables:
    #         table_text = "\n[Table Extracted]\n" + table.df.to_string()
    #         token_count += count_tokens(table_text)
    #         if token_count > token_budget:
    #             print(f"Token budget reached during table extraction: {file_path}")
    #             break
    #         content += table_text
    # except Exception:
    #     pass

    return content, token_count

# --- Stage 3: OCR extraction ---
# def extract_ocr(file_path, token_budget, current_tokens=0):
#     """Extract OCR text (figures/scanned pages) within remaining token budget."""
#     content = ""
#     token_count = current_tokens

#     try:
#         images = convert_from_path(file_path)
#         for image in images:
#             ocr_text = pytesseract.image_to_string(image)
#             token_count += count_tokens(ocr_text)
#             if token_count > token_budget:
#                 print(f"Token budget reached during OCR extraction: {file_path}")
#                 break
#             content += "\n[OCR Extracted]\n" + ocr_text
#     except Exception:
#         pass

    return content

# --- Process all matched papers ---
def process_matched_papers(json_path, pdf_folder="content"):
    """
    Load matched papers from JSON and process them using multi-stage extraction:
    Pass 1: Text + Tables
    Pass 2: OCR (Figures)
    Returns dict mapping filename -> combined extracted content.
    """
    matched_files = load_matched_papers(json_path, pdf_folder)
    text_table_data = {}
    token_usage = {}

    for file_path in matched_files:
        print(f"Extracting text/tables: {os.path.basename(file_path)}")
        content, tokens_used = extract_text_and_tables(file_path, EXTRACTION_BUDGET)
        text_table_data[os.path.basename(file_path)] = content
        token_usage[os.path.basename(file_path)] = tokens_used

    # Return text_table_data directly
    return text_table_data

    # Pass 2: Extract OCR for all files (if budget allows)
    # for file_path in matched_files:
    #     filename = os.path.basename(file_path)
    #     remaining_budget = EXTRACTION_BUDGET - token_usage.get(filename, 0)
    #     if remaining_budget > 0:
    #         print(f"Extracting OCR: {filename} (remaining budget: {remaining_budget})")
    #         ocr_content = extract_ocr(file_path, EXTRACTION_BUDGET, token_usage[filename])
    #         results[filename] = text_table_data[filename] + ocr_content
    #     else:
    #         print(f"Skipping OCR for {filename} (no remaining token budget)")
    #         results[filename] = text_table_data[filename]

# Example usage:
# matched_content = process_matched_papers("/mnt/data/prompt_evaluation_log_cleaned.json", pdf_folder="../content")
# print(matched_content.keys())


In [ ]:
matched_content = process_matched_papers("prompt_evaluation_log_cleaned.json", pdf_folder="data/raw")

In [ ]:
print(matched_content)

In [ ]:
research_proposal_template_prompt = f"""


<WRITE YOUR PROMPT HERE>


"""

In [ ]:
research_plan = llm.invoke(research_proposal_template_prompt)

In [ ]:
display(Markdown(research_plan.content))

In [ ]:
# @title **Optional Part - Creating a PDF of the Research Proposal**
# The code in this cell block is used for printing out the output in the PDF format
from markdown_pdf import MarkdownPdf, Section

pdf = MarkdownPdf()
pdf.add_section(Section(research_plan.content))
pdf.save("Reseach Proposal First Draft.pdf")

> **Note:** The following section explains core functionality and workflow.

## **Step 5: Proposal Evaluation Against NOFO Criteria - [3 Marks]**
> **Use the LLM to evaluate the generated proposal (LLM-as-Judge) and assess its alignment with the NOFO criteria.**
   

---
<font color=Red>**Note:**</font> *2 marks are awarded for the prompt and 1 mark for the successful completion of this section, including debugging or modifying the code if necessary.*

**TASK:** Write an Prompt which can be used to evaluate the Research Proposal based on:
1. **Innovation**
2. **Significance**
3. **Approach**
4. **Investigator Expertise**

- Ask the LLM to rate on each of the criteria from **1 (Poor)** to **5 (Excellent)**
- Ask the LLM to provide the resonse in the json format
```JSON
name: Innovation
    justification: "<Justification>"
    score: <1-5>
    strengths: "<Strength 1>"
    weaknesses: "<Weakness 1>"
    recommendations: "<Recommendation 1>"
```



In [ ]:
evaluation_prompt = f'''


<WRITE YOUR PROMPT HERE>


'''

In [ ]:
# Call the LLM with the prepared prompt and truncated paper content
eval_response = llm.invoke(evaluation_prompt)

In [ ]:
# Import required libraries for core functionality
import json
json_resp = json.loads(eval_response.content[7:-3])

In [ ]:
for key, value in json_resp.items():
  print(f"---\n{key}:")
  if isinstance(value, list):
    for item in value:
      for k, v in item.items():
        print(f"  {k}: {v}")
      print("="*50)
  elif isinstance(value, dict):
    for k, v in value.items():
      print(f"  {k}: {v}")
  else:
    print(f"  {value}")

> **Note:** The following section explains core functionality and workflow.

## **Step 6: Human Review and Refinement of Proposal**
> **Perform Human Evaluation of the generated Proposal. Edit or Modify the proposal as necessary.**

In [ ]:
display(Markdown(research_plan.content))

# **Step 7: Summary and Recommendation - [2 Marks]**


Based on the projects, learners are expected to share their observations, key learnings, and insights related to this business use case, including the challenges they encountered.

Additionally, they should recommend or explain any changes that could improve the project, along with suggesting additional steps that could be taken for further enhancement.



In [ ]:

# --- Enhanced PDF Processing (Commenting original PyPDF-only approach) ---
# Original starter code (commented for traceability):
# Load PDF files and extract content using PyPDFLoader
# docs = PyPDFLoader(file_path, mode="single").load()

# New Implementation: Multi-stage parsing (PyPDF → Camelot/Tabula → OCR fallback)
# Purpose: Capture text, tables, and figures from diverse PDF formats (Mermaid C node, Rubric Step 2).

from PyPDF2 import PdfReader
# Import required libraries for core functionality
import camelot
import pytesseract
from pdf2image import convert_from_path

def process_pdf_multistage(file_path):
    """
    Multi-stage pipeline for extracting text, tables, and figures from PDFs.
    Stages:
    1. PyPDF (text)
    2. Camelot/Tabula (tables)
    3. OCR (scanned pages/figures)
    """
    content = ""

    # Stage 1: PyPDF text extraction
    try:
        reader = PdfReader(file_path)
        for page in reader.pages:
            content += page.extract_text() or ""
    except Exception as e:
        print(f"PyPDF extraction failed: {e}")

    # Stage 2: Table extraction (Camelot)
    try:
        tables = camelot.read_pdf(file_path, pages='all')
        for table in tables:
            content += "\n[Table Extracted]\n" + table.df.to_string()
    except Exception:
        pass

    # Stage 3: OCR fallback for scanned pages or figures
    try:
        images = convert_from_path(file_path)
        for image in images:
            text = pytesseract.image_to_string(image)
            content += "\n[OCR Extracted]\n" + text
    except Exception:
        pass

    return content


In [ ]:

# --- Hybrid Retrieval (BM25 + Embeddings) ---
# Original code used either BM25 OR embeddings; this combines both (Mermaid D node, Rubric Step 2).

from rank_bm25 import BM25Okapi
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings

def hybrid_retrieval_setup(docs_text):
    """
    Creates BM25 and embedding indexes for hybrid search.
    """
    # BM25 Index
    tokenized_corpus = [doc.split(" ") for doc in docs_text]
    bm25 = BM25Okapi(tokenized_corpus)

    # Embedding Index
    embed_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
    vectorstore = Chroma.from_texts(docs_text, embed_model)

    return bm25, vectorstore


In [ ]:

# --- Agentic Components (Research Analyst, Proposal Writer, Compliance Checker) ---
# Implements multi-agent workflow (Mermaid E subgraph, Rubric Step 3-4).

from langchain.agents import initialize_agent, Tool

def analyze_papers(query):
    return "Synthesis of relevant papers"

def check_compliance(proposal):
    return "Compliance report"

tools = [
    Tool(name="Research Analyst", func=analyze_papers, description="Synthesizes relevant papers."),
    Tool(name="Compliance Checker", func=check_compliance, description="Ensures NOFO alignment.")
]

# Initialize agent with zero-shot reasoning and tools
agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)


In [ ]:

# --- Agentic Components (Research Analyst, Proposal Writer, Compliance Checker) ---
# Implements multi-agent workflow (Mermaid E subgraph, Rubric Step 3-4).

from langchain.agents import initialize_agent, Tool

def analyze_papers(query):
    return "Synthesis of relevant papers"

def check_compliance(proposal):
    return "Compliance report"

tools = [
    Tool(name="Research Analyst", func=analyze_papers, description="Synthesizes relevant papers."),
    Tool(name="Compliance Checker", func=check_compliance, description="Ensures NOFO alignment.")
]

# Initialize agent with zero-shot reasoning and tools
agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)


In [ ]:

# --- Multi-Criteria Evaluation with Guardrails ---
# Original evaluation only scored NIH criteria; now adds guardrail flags (Mermaid G node, Rubric Step 5).

evaluation_prompt = f"""
Evaluate the proposal on:
1. Innovation
2. Significance
3. Approach
4. Investigator Expertise

Return JSON:
{{
  "criteria": [
    {{
      "name": "Innovation",
      "score": 1-5,
      "strengths": "...",
      "weaknesses": "...",
      "recommendations": "..."
    }},
    ...
  ],
  "overall_score": 1-5,
  "guardrail_flags": ["hallucination risk", "compliance gap"]
}}
"""


In [ ]:

# --- Caching Intermediate Steps ---
# Saves embeddings, filtered papers, and draft proposals for reuse (Mermaid J node, Rubric Step 7).

# Import required libraries for core functionality
import pickle

def save_checkpoint(data, name):
    with open(f"checkpoint_{name}.pkl", "wb") as f:
        pickle.dump(data, f)

def load_checkpoint(name):
    try:
        with open(f"checkpoint_{name}.pkl", "rb") as f:
            return pickle.load(f)
    except FileNotFoundError:
        return None


> **Note:** The following section explains core functionality and workflow.


# Quick Reference: Few-Shot + Agentic Enhancements

This section provides details about the few-shot pool, semantic versioning, and agentic conflict resolver integrated into this workflow.

---

## Key Features

**Semantic Versioning**
- Automatically increments version numbers (`v2-fewshot`, `v3-agentic`) based on features used.
- Few-shot only → `-fewshot`
- Few-shot + agentic resolver → `-agentic`

**Few-Shot Pool**
- Derived from cleaned log (`prompt_evaluation_log_cleaned.json`).
- Filters examples with ≥80% hybrid confidence.
- Balances relevant/irrelevant examples 50/50 and ensures diversity.

**Agentic Conflict Resolver**
- Activates when model vs. rule confidence differs by >20%.
- Produces reconciled decision and rationale logged under `agentic_resolution`.

**Enhanced Logging Fields**
- `decision_source`: hybrid (model + rule)
- `hybrid_confidence`: average of model and rule confidence
- `agentic_resolution`: reconciliation result (if applicable)
- `prompt_version`: auto-generated semantic version


In [ ]:

# ------------------------------------------------------------
# VERSION TRACKING + FEW-SHOT REBUILDER + AGENTIC RESOLVER
# ------------------------------------------------------------

# Function: Determine the next semantic version string for the prompt
def get_next_prompt_version(log_path, agentic_enabled=False):
    """
    Determine next semantic version based on last logged version.
    Increments number, adds suffix based on features used.
    """
# Import required libraries for core functionality
    import os, json, re
    version_num = 1
    if os.path.exists(log_path):
        with open(log_path, "r", encoding="utf-8") as f:
            try:
                data = json.load(f)
            except json.JSONDecodeError:
                data = []
        # Extract last version number
        for entry in reversed(data):
            if "prompt_version" in entry:
                match = re.match(r"v(\d+)", entry["prompt_version"])
                if match:
                    version_num = int(match.group(1)) + 1
                break

    suffix = "-agentic" if agentic_enabled else "-fewshot"
    return f"v{version_num}{suffix}"


# Function: Build balanced high-confidence few-shot example pool from the log
def rebuild_few_shot_pool(cleaned_log_path, min_conf=80, max_examples=4):
    """
    Build balanced high-confidence few-shot pool from cleaned log.
    Balances relevant and irrelevant, ensures diversity.
    """
# Import required libraries for core functionality
    import json, random
    with open(cleaned_log_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    relevant, irrelevant = [], []
    for iteration in data:
        for doc in iteration.get("relevant_documents", []):
            hybrid_conf = max(doc.get("model_confidence", 0), doc.get("rule_confidence", 0))
            if hybrid_conf >= min_conf:
                relevant.append((doc["title"], doc["reasoning"]))
        for doc in iteration.get("irrelevant_documents", []):
            irrelevant.append((doc, "PAPER NOT RELATED TO TOPIC"))

    # Shuffle and balance
    half = max_examples // 2
    random.shuffle(relevant)
    random.shuffle(irrelevant)
    return relevant[:half] + irrelevant[:half]


# Function: Resolve discrepancies between model and rule confidences using agentic logic
def agentic_conflict_resolver(doc_title, reasoning_json, model_conf, rule_conf):
    """
    Agentic layer to reconcile conflicts:
    - Triggered when discrepancy exceeds threshold
    - Returns reconciled decision and rationale
    """
    rationale = []
    if abs(model_conf - rule_conf) > 20:
        if rule_conf > model_conf:
            final_decision = "RELEVANT" if rule_conf >= 50 else "PAPER NOT RELATED TO TOPIC"
            rationale.append("Rule confidence higher; prioritizing deterministic criteria.")
        else:
            final_decision = "RELEVANT" if model_conf >= 50 else "PAPER NOT RELATED TO TOPIC"
            rationale.append("Model confidence higher; prioritizing LLM interpretation.")
    else:
        final_decision = "RELEVANT" if (model_conf + rule_conf) / 2 >= 50 else "PAPER NOT RELATED TO TOPIC"
        rationale.append("Confidences close; hybrid average used for decision.")

    return {
        "final_decision": final_decision,
        "rationale": " ".join(rationale)
    }


In [ ]:

# ------------------------------------------------------------
# ENHANCED LOGGING WITH SEMANTIC VERSIONING AND AGENTIC RESOLUTION
# ------------------------------------------------------------

# Ensure this cell is run AFTER document processing and building relevant_docs_with_reasoning

# Define constants for few-shot
# Define configuration for few-shot example retrieval (number of examples)
FEW_SHOT_MAX_EXAMPLES = 4
# Minimum confidence threshold for including examples in few-shot prompting
MIN_CONFIDENCE_FOR_FEWSHOT = 70
# Path to the cleaned JSON log file where prompt evaluation iterations are stored
LOG_PATH = "prompt_evaluation_log_cleaned.json"

# Determine prompt version
# Path to the cleaned JSON log file where prompt evaluation iterations are stored
current_version = get_next_prompt_version(LOG_PATH, agentic_enabled=any(doc.get('flagged_for_review', False) for doc in relevant_docs_with_reasoning))

# Add decision source and hybrid confidence
for doc in relevant_docs_with_reasoning:
    doc["decision_source"] = "hybrid"
    doc["hybrid_confidence"] = (doc["model_confidence"] + doc["rule_confidence"]) / 2

# Add agentic resolution for flagged docs
for doc in relevant_docs_with_reasoning:
    if doc.get("flagged_for_review"):
        resolution = agentic_conflict_resolver(
            doc_title=doc["title"],
            reasoning_json=doc["reasoning"],
            model_conf=doc["model_confidence"],
            rule_conf=doc["rule_confidence"]
        )
        doc["agentic_resolution"] = resolution

# Append prompt_version to log
# Path to the cleaned JSON log file where prompt evaluation iterations are stored
with open(LOG_PATH, "r+", encoding="utf-8") as f:
    data = json.load(f)
    if data:
        data[-1]["prompt_version"] = current_version
    f.seek(0)
    json.dump(data, f, indent=2, ensure_ascii=False)
    f.truncate()

print(f"Logged with prompt version: {current_version}")



# Additional enhancements proposed by Claude

Your flowchart shows a well-structured approach to the RFP response generation system. Here are several improvements I'd recommend to enhance the robustness and effectiveness of your solution:

1. Enhanced RFP Requirements Extraction
After step B, add a sub-process for:

Requirement Categorization: Classify requirements into mandatory vs. optional, technical vs. administrative
Scoring Rubric Extraction: Specifically parse how proposals will be evaluated
Budget Constraints Analysis: Extract funding limits and cost-effectiveness criteria
Timeline Extraction: Identify key dates and milestone requirements

2. Improved Paper Processing Pipeline
Between steps C and D, consider adding:

Citation Network Analysis: Map relationships between papers to identify influential work
Method/Innovation Extraction: Specifically extract methodologies and novel approaches
Results/Outcomes Extraction: Capture quantitative results and impact metrics
Quality Assessment: Add a paper quality scoring mechanism (impact factor, recency, relevance)

3. Enhanced Retrieval and Ranking
Expand step D with:

Multi-Query Generation: Generate multiple search queries from different RFP aspects
Cross-Reference Validation: Verify that selected papers actually support proposed innovations
Diversity Scoring: Ensure selected papers cover different aspects of the RFP
Gap Analysis: Identify what the RFP asks for that isn't well-covered in existing research

4. Strengthened Agentic Architecture
Add these specialized agents to your existing three:

Innovation Synthesizer Agent: Combines findings from multiple papers into novel approaches
Budget Estimator Agent: Ensures proposals are financially realistic
Risk Assessment Agent: Identifies potential implementation challenges
Competitive Analysis Agent: Positions your proposal against likely competitors

5. Improved Evaluation and Refinement
Enhance the evaluation loop (G-I) with:

Specific Weakness Detection: Not just overall score, but identify specific weak sections
Competitive Benchmarking: Compare against successful past proposals if available
Consistency Checking: Ensure all sections align and support each other
Technical Feasibility Validation: Verify proposed solutions are implementable

6. Additional Process Improvements
Consider these architectural enhancements:
flowchart LR
    subgraph "Knowledge Management"
        KB1[Domain Ontology]
        KB2[Success Patterns DB]
        KB3[Common Pitfalls DB]
    end
    
    subgraph "Feedback Loops"
        FL1[Real-time Agent Collaboration]
        FL2[Iterative Improvement Tracking]
        FL3[Version Control System]
    end
7. Quality Assurance Additions

Plagiarism Detection: Ensure generated content is original
Fact Verification: Cross-check claims against source papers
Readability Analysis: Ensure proposal meets target audience expectations
Compliance Validation: Automated checks against all RFP requirements

8. Output Enhancement
For the final deliverables (step N), consider generating:

Executive Summary: One-page overview for quick review
Technical Appendix: Detailed methodology descriptions
Budget Justification: Line-by-line cost explanations
Risk Mitigation Plan: Addressing identified challenges
Evaluation Metrics: How success will be measured

9. Monitoring and Logging
Add throughout the pipeline:

Decision Logging: Track why papers were selected/rejected
Agent Reasoning Traces: Understand how proposals were generated
Performance Metrics: Time taken, resources used, quality scores
Error Handling: Graceful degradation if components fail

10. Advanced Features
Consider these stretch goals:

Multi-RFP Learning: Learn from multiple RFPs to improve over time
Collaborative Filtering: If multiple users, learn from collective behavior
Adaptive Prompting: Adjust prompts based on intermediate results
Uncertainty Quantification: Flag areas where the system is less confident
